In [1]:
#トラカンデータの距離案分()
import numpy as np
import psycopg2
import datetime
import pandas as pd

In [96]:
#計測地点のosm_idを指定
start_osm = 266396723
mid_osm = [232919754,651358177]
end_osm = 637167088
start_mesh = 50325662142
end_mesh = 50325652441

In [97]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [98]:
getdbname = "kddi_12_new_param"
dbname = "kddi_trkn12_monthly_anbun_3e_19"
trknName = "trkn_12_grp"
sttime = "2022-04-01 12:00:00"

In [99]:
datedir = "230225_1"
f = open(datedir+'/'+ dbname + '.csv', 'w')
for i in range(30):
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    if i != 0:
        dte = dte + datetime.timedelta(days=i)
    for j in range(24):
        timestr = dte.strftime('%H:%M:%S')
        #特定日付のKDDIメッシュ情報を全取得
        cur =connection.cursor()
        sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM " + getdbname + " "
        sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
        sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
        cur.execute(sql)
        kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
        cur.close()

        one_target_list = []
        one_target_list.append(start_mesh)
        one_target_list.append(end_mesh)
        two_target_list = []
        two_target_list.append(start_osm)
        if len(mid_osm) != 0:
            for target in mid_osm:
                two_target_list.append(target)
        if start_osm != end_osm:
            two_target_list.append(end_osm)

        cur =connection.cursor()
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            stlat = row[0]
            stlng = row[1]
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            edlat = row[0]
            edlng = row[1]
        cur.close()

        if stlat > edlat:
            wklat = stlat
            stlat = edlat
            edlat = wklat
        if stlng > edlng:
            wklng = stlng
            stlng = edlng
            edlng = wklng

        print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))

        keylist = []
        cnt = 0
        for item in two_target_list:
            sql = "SELECT a.grid_code,a.latitude,a.longitude"
            sql += " FROM pointdata a, roadmap r"
            sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
            sql += " order by ST_Distance(a.geom, r.geom);"
            cur =connection.cursor()
            cur.execute(sql)
            childlist = []
            for row in cur:
                if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                    childlist.append([row[0], row[1], row[2]])
                #print(row[0])
            worklist = [item, childlist]
            keylist.append(worklist)
            cnt = cnt + 1
            cur.close()
        #特定日付のトラカンデータを全取得 > 緯度経度
        cur =connection.cursor()
        #sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
        #sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
        #sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
        #sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
        #sql += "ORDER BY trakanmap.gridcode"
        sql = "SELECT grid_code, tbran, latitude, longitude FROM " + trknName  + " "
        sql += "INNER JOIN pointdata ON mesh = cast(grid_code as bigint) "
        sql += "WHERE times = '"+timestr+"' AND (mesh = '"+str(start_mesh)+"' or mesh = '"+str(end_mesh)+"') "
        sql += "ORDER BY mesh"
        #print(sql)
        cur.execute(sql)
        trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
        cur.close()
        trknDf
        #keylist

        from geopy.distance import geodesic

        #データが存在する道路IDごとに交通量を平均化して取得する
        lstItems = []
        cntDt = 0
        wks = ""
        wknp=""
        find = 0

        trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
        trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')
        trknStParam

        if len(trknStParam) == 0:
            print(str(start_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknStParam) == 1:
            trknSt = trknStParam['danmen'].iloc[-1]
        else:
            trknSt = trknStParam['danmen'].iloc[0] + trknStParam['danmen'].iloc[1]

        if len(trknEdParam) == 0:
            print(str(end_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknEdParam) == 1:
            trknEd = trknEdParam['danmen'].iloc[-1]
        else:
            trknEd = trknEdParam['danmen'].iloc[0] + trknEdParam['danmen'].iloc[1]

        st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
        ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]

        dis_km = geodesic(st_station, ed_station)
        #cur =connection.cursor()
        for row in keylist:
            for meshid in row[1]:
                #路線のメッシュ数分ループ
                #print(str(meshid[0]))
                kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                if len(kddiParam) == 0:
                    print(str(meshid[0]) + " is kddi none!")
                    continue
                #路線のメッシュがkddiに存在したなら
                
                kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                kddi_km = geodesic(st_station, kddi_station)
                
                if trknSt < trknEd:
                    ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                else:
                    ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                #sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
                #cur.execute(sql)
                #connection.commit()

        #sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
        f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
        #cur.execute(sql)
        #connection.commit()
        #cur.close()
        dte = dte + datetime.timedelta(minutes=15) 
        #print(lstItems)
f.close()

33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値45.63229955547767距離0.14468426940260257 km
存在メッシュ：50325662142値28.266666666666666距離0.0 km
存在メッシュ：50325662213値50.49009770504899距離0.1851577137438622 km
存在メッシュ：50325662231値45.63229955547767距離0.14468426940260257 km
存在メッシュ：50325662142値28.266666666666666距離0.0 km
存在メッシュ：50325662232値62.997932443614566距離0.2893685387995886 km
存在メッシュ：50325662124値42.1343278926352距離0.11554041512042086 km
存在メッシュ：50325662213値50.49009770504899距離0.1851577137438622 km
存在メッシュ：50325662231値45.63229955547767距離0.14468426940260257 km
存在メッシュ：50325662232値62.997932443614566距離0.2893685387995886 km
存在メッシュ：50325662213値50.49009770504899距離0.1851577137438622 km
存在メッシュ：50325662142値28.266666666666666距離0.0 km
存在メッシュ：50325662232値62.997932443614566距離0.2893685387995886 km
存在メッシュ：50325662231値45.63229955547767距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値47.00197470026938距離0.14468426940260257 km
存在メッシュ：50325662142値30.

33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値47.910731470206784距離0.14468426940260257 km
存在メッシュ：50325662142値29.883333333333333距離0.0 km
存在メッシュ：50325662213値52.95364939157517距離0.1851577137438622 km
存在メッシュ：50325662231値47.910731470206784距離0.14468426940260257 km
存在メッシュ：50325662142値29.883333333333333距離0.0 km
存在メッシュ：50325662232値65.93812960638044距離0.2893685387995886 km
存在メッシュ：50325662124値44.279459967387396距離0.11554041512042086 km
存在メッシュ：50325662213値52.95364939157517距離0.1851577137438622 km
存在メッシュ：50325662231値47.910731470206784距離0.14468426940260257 km
存在メッシュ：50325662232値65.93812960638044距離0.2893685387995886 km
存在メッシュ：50325662213値52.95364939157517距離0.1851577137438622 km
存在メッシュ：50325662142値29.883333333333333距離0.0 km
存在メッシュ：50325662232値65.93812960638044距離0.2893685387995886 km
存在メッシュ：50325662231値47.910731470206784距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値48.219014236935934距離0.14468426940260257 km
存在メッシュ：50325662142

33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値48.86594111835986距離0.14468426940260257 km
存在メッシュ：50325662142値32.94915254237288距離0.0 km
存在メッシュ：50325662213値53.31844489905227距離0.1851577137438622 km
存在メッシュ：50325662231値48.86594111835986距離0.14468426940260257 km
存在メッシュ：50325662142値32.94915254237288距離0.0 km
存在メッシュ：50325662232値64.78272969372897距離0.2893685387995886 km
存在メッシュ：50325662124値45.65981118582336距離0.11554041512042086 km
存在メッシュ：50325662213値53.31844489905227距離0.1851577137438622 km
存在メッシュ：50325662231値48.86594111835986距離0.14468426940260257 km
存在メッシュ：50325662232値64.78272969372897距離0.2893685387995886 km
存在メッシュ：50325662213値53.31844489905227距離0.1851577137438622 km
存在メッシュ：50325662142値32.94915254237288距離0.0 km
存在メッシュ：50325662232値64.78272969372897距離0.2893685387995886 km
存在メッシュ：50325662231値48.86594111835986距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値49.50171120788222距離0.14468426940260257 km
存在メッシュ：50325662142値32.76271

33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値45.305954992302986距離0.14468426940260257 km
存在メッシュ：50325662142値26.93220338983051距離0.0 km
存在メッシュ：50325662213値50.445760555777895距離0.1851577137438622 km
存在メッシュ：50325662231値45.305954992302986距離0.14468426940260257 km
存在メッシュ：50325662142値26.93220338983051距離0.0 km
存在メッシュ：50325662232値63.67970659406221距離0.2893685387995886 km
存在メッシュ：50325662124値41.604917266982525距離0.11554041512042086 km
存在メッシュ：50325662213値50.445760555777895距離0.1851577137438622 km
存在メッシュ：50325662231値45.305954992302986距離0.14468426940260257 km
存在メッシュ：50325662232値63.67970659406221距離0.2893685387995886 km
存在メッシュ：50325662213値50.445760555777895距離0.1851577137438622 km
存在メッシュ：50325662142値26.93220338983051距離0.0 km
存在メッシュ：50325662232値63.67970659406221距離0.2893685387995886 km
存在メッシュ：50325662231値45.305954992302986距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値45.66340181351933距離0.14468426940260257 km
存在メッシュ：50325662142値

33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値48.46762696249857距離0.14468426940260257 km
存在メッシュ：50325662142値31.733333333333334距離0.0 km
存在メッシュ：50325662213値53.14881659414854距離0.1851577137438622 km
存在メッシュ：50325662231値48.46762696249857距離0.14468426940260257 km
存在メッシュ：50325662142値31.733333333333334距離0.0 km
存在メッシュ：50325662232値65.20192059101419距離0.2893685387995886 km
存在メッシュ：50325662124値45.09682641135821距離0.11554041512042086 km
存在メッシュ：50325662213値53.14881659414854距離0.1851577137438622 km
存在メッシュ：50325662231値48.46762696249857距離0.14468426940260257 km
存在メッシュ：50325662232値65.20192059101419距離0.2893685387995886 km
存在メッシュ：50325662213値53.14881659414854距離0.1851577137438622 km
存在メッシュ：50325662142値31.733333333333334距離0.0 km
存在メッシュ：50325662232値65.20192059101419距離0.2893685387995886 km
存在メッシュ：50325662231値48.46762696249857距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値48.60554094721733距離0.14468426940260257 km
存在メッシュ：50325662142値32.05

...

33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値51.776383732435974距離0.14468426940260257 km
存在メッシュ：50325662142値34.0距離0.0 km
存在メッシュ：50325662213値56.749083809329605距離0.1851577137438622 km
存在メッシュ：50325662231値51.776383732435974距離0.14468426940260257 km
存在メッシュ：50325662142値34.0距離0.0 km
存在メッシュ：50325662232値69.5527674641819距離0.2893685387995886 km
存在メッシュ：50325662124値48.19567423788369距離0.11554041512042086 km
存在メッシュ：50325662213値56.749083809329605距離0.1851577137438622 km
存在メッシュ：50325662231値51.776383732435974距離0.14468426940260257 km
存在メッシュ：50325662232値69.5527674641819距離0.2893685387995886 km
存在メッシュ：50325662213値56.749083809329605距離0.1851577137438622 km
存在メッシュ：50325662142値34.0距離0.0 km
存在メッシュ：50325662232値69.5527674641819距離0.2893685387995886 km
存在メッシュ：50325662231値51.776383732435974距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値51.38486017905072距離0.14468426940260257 km
存在メッシュ：50325662142値33.42372881355932距離0.0 km
存在メッシュ：5032566221

33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値46.39864545492556距離0.14468426940260257 km
存在メッシュ：50325662142値28.55距離0.0 km
存在メッシュ：50325662213値51.39155975977404距離0.1851577137438622 km
存在メッシュ：50325662231値46.39864545492556距離0.14468426940260257 km
存在メッシュ：50325662142値28.55距離0.0 km
存在メッシュ：50325662232値64.24729090915824距離0.2893685387995886 km
存在メッシュ：50325662124値42.80338023072062距離0.11554041512042086 km
存在メッシュ：50325662213値51.39155975977404距離0.1851577137438622 km
存在メッシュ：50325662231値46.39864545492556距離0.14468426940260257 km
存在メッシュ：50325662232値64.24729090915824距離0.2893685387995886 km
存在メッシュ：50325662213値51.39155975977404距離0.1851577137438622 km
存在メッシュ：50325662142値28.55距離0.0 km
存在メッシュ：50325662232値64.24729090915824距離0.2893685387995886 km
存在メッシュ：50325662231値46.39864545492556距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値47.91350294672769距離0.14468426940260257 km
存在メッシュ：50325662142値30.7距離0.0 km
存在メッシュ：50325662213値52.728744826

33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値50.71615592244653距離0.14468426940260257 km
存在メッシュ：50325662142値34.53333333333333距離0.0 km
存在メッシュ：50325662213値55.24307907759895距離0.1851577137438622 km
存在メッシュ：50325662231値50.71615592244653距離0.14468426940260257 km
存在メッシュ：50325662142値34.53333333333333距離0.0 km
存在メッシュ：50325662232値66.89897851093153距離0.2893685387995886 km
存在メッシュ：50325662124値47.45643857128693距離0.11554041512042086 km
存在メッシュ：50325662213値55.24307907759895距離0.1851577137438622 km
存在メッシュ：50325662231値50.71615592244653距離0.14468426940260257 km
存在メッシュ：50325662232値66.89897851093153距離0.2893685387995886 km
存在メッシュ：50325662213値55.24307907759895距離0.1851577137438622 km
存在メッシュ：50325662142値34.53333333333333距離0.0 km
存在メッシュ：50325662232値66.89897851093153距離0.2893685387995886 km
存在メッシュ：50325662231値50.71615592244653距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値48.993911567550654距離0.14468426940260257 km
存在メッシュ：50325662142値32.3166

33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値44.99717763482269距離0.14468426940260257 km
存在メッシュ：50325662142値27.966101694915253距離0.0 km
存在メッシュ：50325662213値49.76138805639607距離0.1851577137438622 km
存在メッシュ：50325662231値44.99717763482269距離0.14468426940260257 km
存在メッシュ：50325662142値27.966101694915253距離0.0 km
存在メッシュ：50325662232値62.028253574069距離0.2893685387995886 km
存在メッシュ：50325662124値41.56659601381542距離0.11554041512042086 km
存在メッシュ：50325662213値49.76138805639607距離0.1851577137438622 km
存在メッシュ：50325662231値44.99717763482269距離0.14468426940260257 km
存在メッシュ：50325662232値62.028253574069距離0.2893685387995886 km
存在メッシュ：50325662213値49.76138805639607距離0.1851577137438622 km
存在メッシュ：50325662142値27.966101694915253距離0.0 km
存在メッシュ：50325662232値62.028253574069距離0.2893685387995886 km
存在メッシュ：50325662231値44.99717763482269距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値44.84962512417563距離0.14468426940260257 km
存在メッシュ：50325662142値27.88333333

33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値49.26485548597766距離0.14468426940260257 km
存在メッシュ：50325662142値31.716666666666665距離0.0 km
存在メッシュ：50325662213値54.17372115880333距離0.1851577137438622 km
存在メッシュ：50325662231値49.26485548597766距離0.14468426940260257 km
存在メッシュ：50325662142値31.716666666666665距離0.0 km
存在メッシュ：50325662232値66.81304430460744距離0.2893685387995886 km
存在メッシュ：50325662124値45.73011145348638距離0.11554041512042086 km
存在メッシュ：50325662213値54.17372115880333距離0.1851577137438622 km
存在メッシュ：50325662231値49.26485548597766距離0.14468426940260257 km
存在メッシュ：50325662232値66.81304430460744距離0.2893685387995886 km
存在メッシュ：50325662213値54.17372115880333距離0.1851577137438622 km
存在メッシュ：50325662142値31.716666666666665距離0.0 km
存在メッシュ：50325662232値66.81304430460744距離0.2893685387995886 km
存在メッシュ：50325662231値49.26485548597766距離0.14468426940260257 km
33.7984375000:33.8026041665:132.780468750:132.780468750
存在メッシュ：50325662231値49.33779347809222距離0.14468426940260257 km
存在メッシュ：50325662142値31.55

In [100]:
connection.close()

In [70]:
#CreateGeojsonFile